# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
#import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt

# Loading the data

In [ ]:
elements = ["Fe","Mn", "Ni", "Co", "Cr"] #"V", "Mo", "Si", "Al"
savename = 'Input.csv'

In [ ]:
comp = pd.DataFrame(pd.read_excel(r"/content/Composition Data.xlsx"))
comp = comp.drop_duplicates()

prop = pd.DataFrame(pd.read_excel(r"/content/Properties Data.xlsx"))
prop = prop.drop_duplicates()

# Basic Exploration

In [ ]:
comp_columns = comp.shape[1]
print("Number of columns:", comp_columns)

Number of columns: 8


In [ ]:
len(prop)

18

In [ ]:
len(comp)

3701

In [ ]:
prop.head()

,Property,Fe,Mn,Ni,Co,Cr,V,Mo,Si,Al
0,VASP-PAW-GGA LSE,-7.76,-3.01,2.13,-1.99,0.91,0.53,1.14,-3.260,2.85
1,SGTE LSE,-2.24,-1.00,2.89,-0.43,-2.85,-3.50,-3.65,-1.800,5.48
2,Allen EN,1.80,1.75,1.88,1.84,1.65,1.53,1.47,1.916,1.61
3,Pauling EN,1.83,1.55,1.91,1.88,1.66,1.63,2.16,1.900,1.61
4,Density,7.87,7.43,8.90,8.86,7.19,6.11,10.20,2.330,2.70


In [ ]:
prop.shape

(18, 10)

In [ ]:
comp.shape

(3701, 8)

In [ ]:
comp.head()

,Fe,Mn,Co,Cr,Ni,del_G,del_H,Cp_FCC
0,20,20,60,0,0,-494.135792,673.733798,39.221444
1,20,25,55,0,0,-786.373657,-307.880046,25.528730
2,20,30,50,0,0,-767.298921,-300.574605,25.599116
3,20,35,45,0,0,-734.739587,-265.447435,25.697948
4,20,40,40,0,0,-689.236470,-215.159456,25.779076


In [ ]:
comp = comp[['Fe', 'Mn', 'Ni','Co','Cr','del_G','del_H','Cp_FCC']]
comp.head()

,Fe,Mn,Ni,Co,Cr,del_G,del_H,Cp_FCC
0,20,20,0,60,0,-494.135792,673.733798,39.221444
1,20,25,0,55,0,-786.373657,-307.880046,25.528730
2,20,30,0,50,0,-767.298921,-300.574605,25.599116
3,20,35,0,45,0,-734.739587,-265.447435,25.697948
4,20,40,0,40,0,-689.236470,-215.159456,25.779076


In [ ]:
mat_properties = pd.DataFrame(pd.read_excel(r"/content/Properties Data.xlsx", index_col ='Property'))
property_names = mat_properties.index.to_list()

In [ ]:
#prop = sfe.reset_index().copy()))
mat_properties.head()

,Fe,Mn,Ni,Co,Cr,V,Mo,Si,Al
Property,,,,,,,,,
VASP-PAW-GGA LSE,-7.76,-3.01,2.13,-1.99,0.91,0.53,1.14,-3.260,2.85
SGTE LSE,-2.24,-1.00,2.89,-0.43,-2.85,-3.50,-3.65,-1.800,5.48
Allen EN,1.80,1.75,1.88,1.84,1.65,1.53,1.47,1.916,1.61
Pauling EN,1.83,1.55,1.91,1.88,1.66,1.63,2.16,1.900,1.61
Density,7.87,7.43,8.90,8.86,7.19,6.11,10.20,2.330,2.70


In [ ]:
mat_properties = mat_properties.drop(['V','Mo','Si','Al'],axis=1)
mat_properties.head()

,Fe,Mn,Ni,Co,Cr
Property,,,,,
VASP-PAW-GGA LSE,-7.76,-3.01,2.13,-1.99,0.91
SGTE LSE,-2.24,-1.00,2.89,-0.43,-2.85
Allen EN,1.80,1.75,1.88,1.84,1.65
Pauling EN,1.83,1.55,1.91,1.88,1.66
Density,7.87,7.43,8.90,8.86,7.19


In [ ]:
property_names = mat_properties.index.to_list()
property_names

['VASP-PAW-GGA LSE',
 'SGTE LSE',
 'Allen EN',
 'Pauling EN',
 'Density',
 'Atomic Weight',
 'Valence e',
 'IE',
 'Melting Point',
 'Specific Heat',
 'Metallic Radius',
 'Shear Modulus',
 'Total Electrons',
 'Atomic Planar Density',
 'C_11',
 'C_12',
 "C'",
 'Heat of fusion']

# Calculating average and variance of each property for different

In [ ]:
for k in range(len(property_names)):
    # Initialize columns in dataframe for property average and property variance
    comp[property_names[k]+'_Avg'] = 0.0
    comp[property_names[k]+'_Var'] = 0.0

    el_properties = np.array(mat_properties.loc[property_names[k], elements], dtype=np.float64)
    #print(el_properties)

    # Iterate over each row in the dataframe
    for i in range(comp.shape[0]):
        # Get composition of row i
        comp_row = comp.iloc[i][elements].values
        # Compute composition-weighted average of property
        prop_avg = np.dot(comp_row, el_properties)
        # Compute composition-weighted variance of property
        prop_var = np.sqrt(np.dot(comp_row, (el_properties - prop_avg) ** 2))


        comp.at[i,property_names[k]+'_Avg'] = prop_avg
        comp.at[i,property_names[k]+'_Var'] = prop_var

for k in range(len(property_names)):
    comp[property_names[k] + '_Avg'] = comp[property_names[k] + '_Avg'].astype(np.float64)
    comp[property_names[k] + '_Var'] = comp[property_names[k] + '_Var'].astype(np.float64)

# Save new dataframe in csv format

In [ ]:
 comp.to_csv(savename)